# Advent of Haystack - Day 7
Santa collapsed in his chair in a huff. "What's wrong?" asked Mrs Claus.

"There's just too many toys to check and not enough time! Christmas is almost here!"

"Well can't you just check some of them?"

"I wish it were that easy! But my elves make so many different toys, and we have to make sure every kid gets the right one!"

Elf Jane couldn't help overhearing from the next room. She was a regular attendee at the local north pole hackathon, and thought she might have a solution. She'd learned a lot about evaluation recently, and thought she could build an LLM Judge to help.

**For this challenge, you need to help Elf Jane and complete the code cells with `#TODO` text**

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-1.jpeg?raw=true' width=500px>

## Installation

In [ ]:
! pip install -q arize-phoenix==6.1.0 haystack-ai==2.7.0 openinference-instrumentation-haystack==0.1.13 'httpx<0.28'

## Data

Elf Jane started by checking out the big elf database of christmas wishlists (aka the BEDCW).

In [1]:
children = [
    {'name': 'Timmy', 'age': 7, 'likes': 'Lego', 'dislikes': 'Vegetables', 'list': 'nice'},
    {'name': 'Tommy', 'age': 9, 'likes': 'Sports Equipment', 'dislikes': 'Reading', 'list': 'naughty'},
    {'name': 'Tammy', 'age': 8, 'likes': 'Art Supplies', 'dislikes': 'Loud Noises', 'list': 'nice'},
    {'name': 'Tina', 'age': 6, 'likes': 'Science Kits', 'dislikes': 'Spicy Food', 'list': 'nice'},
    {'name': 'Toby', 'age': 10, 'likes': 'Video Games', 'dislikes': 'Early Mornings', 'list': 'nice'},
    {'name': 'Tod', 'age': 5, 'likes': 'Musical Instruments', 'dislikes': 'Bath Time', 'list': 'nice'},
    {'name': 'Todd', 'age': 8, 'likes': 'Remote Control Cars', 'dislikes': 'Homework', 'list': 'naughty'},
    {'name': 'Tara', 'age': 7, 'likes': 'Magic Sets', 'dislikes': 'Thunder', 'list': 'nice'},
    {'name': 'Teri', 'age': 9, 'likes': 'Building Blocks', 'dislikes': 'Broccoli', 'list': 'nice'},
    {'name': 'Trey', 'age': 6, 'likes': 'Board Games', 'dislikes': 'Bedtime', 'list': 'nice'},
    {'name': 'Tyler', 'age': 8, 'likes': 'Action Figures', 'dislikes': 'Cleaning', 'list': 'nice'},
    {'name': 'Tracy', 'age': 7, 'likes': 'Dolls', 'dislikes': 'Dark', 'list': 'nice'},
    {'name': 'Tony', 'age': 9, 'likes': 'Chemistry Sets', 'dislikes': 'Dentist', 'list': 'nice'},
    {'name': 'Theo', 'age': 6, 'likes': 'Puzzles', 'dislikes': 'Shots', 'list': 'nice'},
    {'name': 'Terry', 'age': 10, 'likes': 'Model Trains', 'dislikes': 'Chores', 'list': 'naughty'},
    {'name': 'Tessa', 'age': 5, 'likes': 'Stuffed Animals', 'dislikes': 'Time Out', 'list': 'nice'},
    {'name': 'Troy', 'age': 8, 'likes': 'Robots', 'dislikes': 'Naps', 'list': 'nice'},
    {'name': 'Talia', 'age': 7, 'likes': 'Craft Kits', 'dislikes': 'Spinach', 'list': 'nice'},
    {'name': 'Tyson', 'age': 9, 'likes': 'Microscopes', 'dislikes': 'Cold', 'list': 'nice'},
    {'name': 'Tatum', 'age': 6, 'likes': 'Drawing Sets', 'dislikes': 'Shots', 'list': 'nice'},
]

# 1. Adding Tracing 📝

Elf Jane knew that the elves were busy, and didn't always log their toy making process. She knew that she'd first need to trace the toy making process using Arize Phoenix.

In [2]:
%load_ext dotenv
%dotenv

import os
from phoenix.otel import register
from openinference.instrumentation.haystack import HaystackInstrumentor

ARIZE_API_KEY = os.getenv("ARIZE_API_KEY")
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={ARIZE_API_KEY}"

tracer_provider = register(
  project_name="haystack-example",
  endpoint="https://app.phoenix.arize.com/v1/traces",
)

HaystackInstrumentor().instrument(tracer_provider=tracer_provider)

/opt/homebrew/Caskroom/miniforge/base/envs/arize/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: haystack-example
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# 2. Trace Toy Making Process 🚂

With tracing in place, Elf Jane had some of her closest elf friends build a batch of toys she could trace.

⭐️ Feel free to replace `OpenAIChatGenerator` with other [ChatGenerators](https://docs.haystack.deepset.ai/docs/generators) supported in Haystack

In [30]:
#import os

#os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [4]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.components.builders import ChatPromptBuilder
from haystack import Pipeline
from haystack.utils import Secret

messages = [
    ChatMessage.from_system("You are a toy maker elf. Your job is to make toys for the nice kids on the nice list. If the child is on the naughty list, give them a 'Rabbit R1'. {{name}} is on the {{list}} list"),
    ChatMessage.from_user("Create a toy for {{name}} that they will like. {{name}} is {{age}} years old and likes {{likes}} and dislikes {{dislikes}}."),
]

builder = ChatPromptBuilder(messages)

chat_generator = HuggingFaceAPIChatGenerator(api_type="serverless_inference_api",
                                             api_params={"model": "mistralai/Mistral-Nemo-Instruct-2407"},
                                             token=Secret.from_token(huggingface_api_key))
pipeline = Pipeline()
pipeline.add_component("builder", builder)
pipeline.add_component("chat_generator", chat_generator)

pipeline.connect("builder", "chat_generator")

def make_toy(child):
    return pipeline.run({"builder":{**child}})["chat_generator"]["replies"]

In [5]:
for child in children:
    make_toy(child)

# 3. Evaluate Toy Correctness 🔬

Elf Jane was now ready to evaluate the toys she made. She knew that she could use an LLM Judge to evaluate whether the toys matched the child's wishlist. She started by building a judge.

In [6]:
llm_judge_prompt = """
Evaluate the toy for this child, based on their likes and dislikes

All children on the naughty list get a 'Rabbit R1'. Any other toy given to a naughty child is incorrect.

Respond with a single word: 'correct' or 'incorrect'. Also include a short explanation for your answer.

Description of the child: {description}
Toy: {toy}

*****
Example output:
label: 'correct'
explanation: 'The toy is a Lego set, which is one of the child's likes.'
*****
"""

In [11]:
import phoenix as px
spans_df = px.Client(endpoint="https://app.phoenix.arize.com").get_spans_dataframe(project_name="haystack-example")
spans_df.head()

/opt/homebrew/Caskroom/miniforge/base/envs/arize/lib/python3.10/site-packages/phoenix/utilities/client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/arize/lib/python3.10/site-packages/phoenix/utilities/client.py:45: UserWarning: ⚠️⚠️ The Phoenix server (7.3.0) and client (6.1.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.output.value,attributes.input.value,attributes.input.mime_type,attributes.output.mime_type,attributes.llm.output_messages,attributes.llm.model_name,attributes.llm.token_count.prompt,attributes.llm.token_count.total,attributes.llm.input_messages,attributes.llm.token_count.completion
context.span_id,,,,,,,,,,,,,,,,,,,,,
b286de708b0e2acd,haystack.tracing.auto_enable,UNKNOWN,None,2024-12-27 18:15:29.846167+00:00,2024-12-27 18:15:29.849845+00:00,ERROR,ImportError: cannot import name 'Span' from pa...,"[{'name': 'exception', 'timestamp': '2024-12-2...",b286de708b0e2acd,28dd36e80e0bd199d72c3b47b347fe64,...,None,None,None,None,None,None,NaN,NaN,None,NaN
c9cd1537b590590a,haystack.tracing.auto_enable,UNKNOWN,None,2024-12-27 18:15:31.379916+00:00,2024-12-27 18:15:31.379952+00:00,UNSET,,[],c9cd1537b590590a,90e8e61d76c6bea59e4f18dc5a4fc9f1,...,None,None,None,None,None,None,NaN,NaN,None,NaN
cfda9d374cd114b6,haystack.component.run,UNKNOWN,80d41348ab7c708c,2024-12-27 18:15:40.951228+00:00,2024-12-27 18:15:40.952328+00:00,UNSET,,[],cfda9d374cd114b6,129a3c744c9da11635fb20f4132c34e7,...,None,None,None,None,None,None,NaN,NaN,None,NaN
80d41348ab7c708c,ChatPromptBuilder (builder),CHAIN,bd0f139cddaa8f22,2024-12-27 18:15:40.951133+00:00,2024-12-27 18:15:41.084785+00:00,OK,,[],80d41348ab7c708c,129a3c744c9da11635fb20f4132c34e7,...,"{""prompt"": [""ChatMessage(content=\""You are a t...","{""template"": null, ""template_variables"": null,...",application/json,application/json,None,None,NaN,NaN,None,NaN
06f443a5b24d8c60,haystack.component.run,UNKNOWN,6d221f4b87b80cec,2024-12-27 18:15:41.249755+00:00,2024-12-27 18:15:41.712119+00:00,UNSET,,[],06f443a5b24d8c60,129a3c744c9da11635fb20f4132c34e7,...,None,None,None,None,None,None,NaN,NaN,None,NaN


In [ ]:
eval_df = spans_df[spans_df.name == "Pipeline"][["attributes.input.value","attributes.output.value"]]

In [77]:
import json

In [82]:
eval_df["toy"] = eval_df["attributes.output.value"].apply(lambda x: json.loads(x)["chat_generator"]["replies"][0])
eval_df["description"] = eval_df["attributes.input.value"].apply(lambda x: json.loads(x)["data"]["builder"])

In [83]:
eval_df

,attributes.input.value,attributes.output.value,toy,description
context.span_id,,,,
07d2ea53a30a3df1,"{""data"": {""builder"": {""name"": ""Timmy"", ""age"": ...","{""chat_generator"": {""replies"": [""ChatMessage(c...",ChatMessage(content='I\'m delighted to help cr...,"{'name': 'Timmy', 'age': 7, 'likes': 'Lego', '..."
f193acc70d69f297,"{""data"": {""builder"": {""name"": ""Tommy"", ""age"": ...","{""chat_generator"": {""replies"": [""ChatMessage(c...","ChatMessage(content=""Given Tommy's age and int...","{'name': 'Tommy', 'age': 9, 'likes': 'Sports E..."
6ddf6738a8f127f4,"{""data"": {""builder"": {""name"": ""Tammy"", ""age"": ...","{""chat_generator"": {""replies"": [""ChatMessage(c...",ChatMessage(content=' Presents are magical! An...,"{'name': 'Tammy', 'age': 8, 'likes': 'Art Supp..."
7c9ba4b9ccff5407,"{""data"": {""builder"": {""name"": ""Tina"", ""age"": 6...","{""chat_generator"": {""replies"": [""ChatMessage(c...","ChatMessage(content='**Toy Name:** ""Tina\'s Ma...","{'name': 'Tina', 'age': 6, 'likes': 'Science K..."
aa8b92af314a6fd6,"{""data"": {""builder"": {""name"": ""Toby"", ""age"": 1...","{""chat_generator"": {""replies"": [""ChatMessage(c...",ChatMessage(content='Hello there! As a toy mak...,"{'name': 'Toby', 'age': 10, 'likes': 'Video Ga..."
eef33ce8b42e5b87,"{""data"": {""builder"": {""name"": ""Tod"", ""age"": 5,...","{""chat_generator"": {""replies"": [""ChatMessage(c...","ChatMessage(content='**Toy Name:** ""Tod\'s Tun...","{'name': 'Tod', 'age': 5, 'likes': 'Musical In..."
48a11534f1195c5f,"{""data"": {""builder"": {""name"": ""Todd"", ""age"": 8...","{""chat_generator"": {""replies"": [""ChatMessage(c...",ChatMessage(content='If Todd enjoys Remote Con...,"{'name': 'Todd', 'age': 8, 'likes': 'Remote Co..."
db13a3664f67e174,"{""data"": {""builder"": {""name"": ""Tara"", ""age"": 7...","{""chat_generator"": {""replies"": [""ChatMessage(c...","ChatMessage(content=""Hello there, little elf h...","{'name': 'Tara', 'age': 7, 'likes': 'Magic Set..."
d1e0d16b1ec570b1,"{""data"": {""builder"": {""name"": ""Teri"", ""age"": 9...","{""chat_generator"": {""replies"": [""ChatMessage(c...","ChatMessage(content=""It seems like Teri has a ...","{'name': 'Teri', 'age': 9, 'likes': 'Building ..."


In [84]:
from phoenix.evals import (
    llm_classify,
    OpenAIModel,
    ClassificationTemplate
)

rails = ["correct", "wrong"]
llm = OpenAIModel(model='gpt-4o-mini', api_key=openai_api_key)
template = ClassificationTemplate(
    template=llm_judge_prompt, rails=rails, 
    explanation_template="Explaination: "
    )

eval_results = llm_classify(
    dataframe=eval_df,
    template=template,
    model=llm,
    rails=rails,
)
eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)
eval_results.head()

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_classify |██████████| 20/20 (100.0%) | ⏳ 00:10<00:00 |  1.94it/s


,label,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,
07d2ea53a30a3df1,correct,[],COMPLETED,1.573753,1
f193acc70d69f297,correct,[],COMPLETED,0.408298,1
6ddf6738a8f127f4,correct,[],COMPLETED,0.330364,1
7c9ba4b9ccff5407,correct,[],COMPLETED,0.397995,1
aa8b92af314a6fd6,correct,[],COMPLETED,0.532520,1


In [87]:
from phoenix.trace import SpanEvaluations

px.Client(endpoint="https://app.phoenix.arize.com").log_evaluations(SpanEvaluations(eval_name="Correctness", dataframe=eval_results))


/opt/homebrew/Caskroom/miniforge/base/envs/arize/lib/python3.10/site-packages/phoenix/utilities/client.py:34: UserWarning: The Phoenix server has an unknown version and may have compatibility issues.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/arize/lib/python3.10/site-packages/phoenix/utilities/client.py:45: UserWarning: ⚠️⚠️ The Phoenix server (7.3.0) and client (6.1.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


# 4. View the results in the Arize Phoenix UI 🐦‍🔥

And just like that, Elf Jane had saved Santa hours of time and made sure every kid got the right toy!

In Phoenix, she could see "correct" and "incorrect" labels on all the traces, and even see the explanations for each label!

She couldn't wait to show Santa, and all her friends at the hackathon.

 <img src='https://github.com/Jgilhuly/phoenix-assets/blob/main/images/socal/advent-of-haystack-2.jpeg?raw=true' width=500px>